# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_cleaned_df = pd.read_csv("Resources/cities_weather.csv", encoding='utf-8')
cities_cleaned_df.head()

,City:,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.8000,-68.3000,40.66,87.0,40.0,32.21,AR,1651181945
1,sao jose do belmonte,-7.8614,-38.7597,78.76,59.0,78.0,14.99,BR,1651182170
2,upernavik,72.7868,-56.1549,15.98,87.0,39.0,3.85,GL,1651182170
3,rikitea,-23.1203,-134.9692,77.20,65.0,31.0,20.31,PF,1651182061
4,novodolinskiy,49.7050,72.7047,39.78,40.0,94.0,6.13,KZ,1651182171


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)
coordinates = cities_cleaned_df[['Lat','Lng']]
fig = gmaps.figure()
humidity = cities_cleaned_df['Humidity']
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=True, max_intensity=100, point_radius=10)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
ideal_weather_df = cities_cleaned_df.loc[
    (cities_cleaned_df['Max Temp'] > 70) &
    (cities_cleaned_df['Max Temp'] < 80) & 
    (cities_cleaned_df['Wind Speed'] < 10) &
    (cities_cleaned_df['Cloudiness'] == 0)
]
hotel_df = ideal_weather_df.dropna()
hotel_df

,City:,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,rock sound,24.9000,-76.2000,74.71,61.0,0.0,4.65,BS,1651182209
119,quatre cocos,-20.2078,57.7625,74.08,80.0,0.0,4.72,MU,1651182218
171,grand gaube,-20.0064,57.6608,74.37,82.0,0.0,1.99,MU,1651182239
179,buin,-33.7333,-70.7500,71.87,45.0,0.0,3.11,CL,1651182242
213,laguna,38.4210,-121.4238,72.10,35.0,0.0,5.75,US,1651182255
282,pisco,-13.7000,-76.2167,71.65,68.0,0.0,9.22,PE,1651182283
291,elizabethton,36.3487,-82.2107,71.76,37.0,0.0,4.61,US,1651182128
299,caravelas,-17.7125,-39.2481,75.78,83.0,0.0,4.63,BR,1651182289
351,golaghat,26.5167,93.9667,74.03,66.0,0.0,1.63,IN,1651182314
364,cap malheureux,-19.9842,57.6142,74.46,82.0,0.0,3.00,MU,1651182320


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_names = []
Lats = hotel_df['Lat'].tolist()
Lngs = hotel_df['Lng'].tolist()

for index in range(len(Lats)):
    target_coordinates = f"{str(Lats[index])},{str(Lngs[index])}"
    target_type = "hotel"
    target_radius = 8000
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params)
        results = response.json()
        hotel_names.append(results['results'][0]['name'])
    except:
        hotel_names.append(None)

hotel_df['Hotel Name'] = hotel_names
    

In [47]:
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.rename(columns={
    'City:' : 'City'
})
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
97,rock sound,24.9000,-76.2000,74.71,61.0,0.0,4.65,BS,1651182209,Rock Sound
119,quatre cocos,-20.2078,57.7625,74.08,80.0,0.0,4.72,MU,1651182218,Central Flacq
171,grand gaube,-20.0064,57.6608,74.37,82.0,0.0,1.99,MU,1651182239,Grand Baie
179,buin,-33.7333,-70.7500,71.87,45.0,0.0,3.11,CL,1651182242,San Bernardo
213,laguna,38.4210,-121.4238,72.10,35.0,0.0,5.75,US,1651182255,Sacramento
282,pisco,-13.7000,-76.2167,71.65,68.0,0.0,9.22,PE,1651182283,Pisco
291,elizabethton,36.3487,-82.2107,71.76,37.0,0.0,4.61,US,1651182128,Johnson City
299,caravelas,-17.7125,-39.2481,75.78,83.0,0.0,4.63,BR,1651182289,Caravelas
351,golaghat,26.5167,93.9667,74.03,66.0,0.0,1.63,IN,1651182314,Golaghat
364,cap malheureux,-19.9842,57.6142,74.46,82.0,0.0,3.00,MU,1651182320,Grand Baie


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))